# Scikit-Learn Iris Model using customData

* Wrap a scikit-learn python model for use as a prediction microservice in seldon-core
    * Run locally on Docker to test
    * Deploy on seldon-core running on a Kubernetes cluster

## Dependencies

* [s2i](https://github.com/openshift/source-to-image)
* Seldon Core v1.0.3+ installed
* `pip install sklearn seldon-core protobuf grpcio`

In [ ]:
# !pip3 install sklearn seldon-core protobuf grpcio 
# !pip3 install grpcio-tools
# !pip3 install --upgrade protobuf
# !pip3 install --upgrade pip
!pip install -r ./custom-models/iris/requirements.txt
!pip install --upgrade pip
# !pip install --upgrade protobuf

In [ ]:
# !pip3 uninstall protobuf
# !pip3 install protobuf==4.21

## Train locally

In [2]:
import os

import numpy as np
from sklearn import datasets
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


def main():
    clf = LogisticRegression()
    p = Pipeline([("clf", clf)])
    print("Training model...")
    p.fit(X, y)
    print("Model trained!")

    filename_p = "./custom-models/iris/IrisClassifier.sav"
    print("Saving model in %s" % filename_p)
    joblib.dump(p, filename_p)
    print("Model saved!")


if __name__ == "__main__":
    print("Loading iris data set...")
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    print("Dataset loaded!")
    main()

Loading iris data set...
Dataset loaded!
Training model...
Model trained!
Saving model in ./custom-models/iris/IrisClassifier.sav
Model saved!


/Users/dileep.gadiraju/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Custom Protobuf Specification

First, we'll need to define our custom protobuf specification so that it can be leveraged.

In [4]:
%%writefile ./custom-models/iris/iris.proto

syntax = "proto3";

package iris;

message IrisPredictRequest {
    float sepal_length = 1;
    float sepal_width = 2;
    float petal_length = 3;
    float petal_width = 4;
}

message IrisPredictResponse {
    float setosa = 1;
    float versicolor = 2;
    float virginica = 3;
}

Overwriting ./custom-models/iris/iris.proto


## Custom Protobuf Compilation

We will need to compile our custom protobuf for python so that we can unpack the `customData` field passed to our `predict` method later on.

In [5]:
!python -m grpc.tools.protoc --python_out=./custom-models/iris --proto_path=./custom-models/iris iris.proto

## gRPC test

Wrap model using s2i

In [6]:
# !s2i build ./custom-models/iris seldonio/seldon-core-s2i-python37-ubi8:1.7.0-dev seldonio/sklearn-iris-customdata:0.1
!docker build ./custom-models/iris -t seldonio/sklearn-iris-customdata:0.1

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/2)                                                         
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
[+] Building 0.2s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.7-slim         0.0s
[+] Building 0.4s (2/3)              

Serve the model locally

In [13]:
!docker run --name "iris_predictor" -d --rm -p 5001:5001 seldonio/sklearn-iris-customdata:0.1

329e9bcbe7e8754e03e6112f386aff33def9f640d12a616fcc1c0fa7beeb6b2e


In [ ]:
# import sys,os
# !export PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python
# for item, value in os.environ.items():
#     print('{}: {}'.format(item, value))

Test using custom protobuf payload

In [15]:
!export PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python
import grpc
import sys,os
sys.path.append(os.path.expanduser('./custom-models/iris'))
from iris_pb2 import IrisPredictRequest, IrisPredictResponse

from seldon_core.proto import prediction_pb2, prediction_pb2_grpc

channel = grpc.insecure_channel("localhost:5001")
stub = prediction_pb2_grpc.ModelStub(channel)

iris_request = IrisPredictRequest(
    sepal_length=7.233, sepal_width=4.652, petal_length=7.39, petal_width=0.324
)

seldon_request = prediction_pb2.SeldonMessage()
seldon_request.customData.Pack(iris_request)

response = stub.Predict(seldon_request)

iris_response = IrisPredictResponse()
response.customData.Unpack(iris_response)

print(iris_response)

setosa: 3.1866779863776173e-06
versicolor: 0.11372585594654083
virginica: 0.8862709403038025



Stop serving model

In [21]:
!docker rm iris_predictor --force

iris_predictor


## Setup Seldon Core

Use the [setup notebook](https://github.com/SeldonIO/seldon-core/blob/master/notebooks/seldon_core_setup.ipynb) to setup Seldon Core with an ingress - either Ambassador or Istio

Then port-forward to that ingress on localhost:8004 in a separate terminal either with:

* Ambassador: `kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8004:8080`
* Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8004:80`

In [16]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [17]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-seldon" modified.


## Deploy your Seldon Model

We first create a configuration file:

In [18]:
%%writefile sklearn_iris_customdata_deployment.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-deployment-example
spec:
  name: sklearn-iris-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: groszewn/sklearn-iris-customdata:0.1
          imagePullPolicy: IfNotPresent
          name: sklearn-iris-classifier
    graph:
      children: []
      endpoint:
        type: GRPC
      name: sklearn-iris-classifier
      type: MODEL
    name: sklearn-iris-predictor
    replicas: 1

Writing sklearn_iris_customdata_deployment.yaml


### Run the model in our cluster

Apply the Seldon Deployment configuration file we just created

In [19]:
!kubectl create -f sklearn_iris_customdata_deployment.yaml

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


### Check that the model has been deployed

In [20]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-deployment-example -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "seldon-92a927e5e90d7602e08ba9b9304f70e8" rollout to finish: 0 of 1 updated replicas are available...
^C


## Test by sending prediction calls

`IrisPredictRequest` sent via the `customData` field.

In [ ]:
iris_request = IrisPredictRequest(
    sepal_length=7.233, sepal_width=4.652, petal_length=7.39, petal_width=0.324
)

seldon_request = prediction_pb2.SeldonMessage()
seldon_request.customData.Pack(iris_request)

channel = grpc.insecure_channel("localhost:8004")
stub = prediction_pb2_grpc.SeldonStub(channel)

metadata = [("seldon", "seldon-deployment-example"), ("namespace", "seldon")]

response = stub.Predict(request=seldon_request, metadata=metadata)

iris_response = IrisPredictResponse()
response.customData.Unpack(iris_response)

print(iris_response)

### Cleanup our deployment

In [22]:
!kubectl delete -f sklearn_iris_customdata_deployment.yaml

seldondeployment.machinelearning.seldon.io "seldon-deployment-example" deleted
